# Preparing point data for the atlas website

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_preparing_point_data.ipynb)

To upload point based data to the website you will need to provide
- A `points` array of 3D coordinates for each point to be displayed.  
- One (or more) `feature` array(s) containing the values to be displayed, each corresponding to a point in the  listed in the `points` array
- The names of each features


## Installation

In [ ]:
!pip install iblbrainviewer

## Preparing your points

### Defining the coordinates 

For each point you must provide the 3D location in the following coordinate framework,

 - In metres
 - Relative to bregma
 - Order ml, ap, dv

This is the standard coordinate framework used within the International brain laboratory.

If you have coordinates that are in the Allen CCF coordinate framework, the following code can be used to transform your coordinates.

In [1]:
from iblatlas.atlas import AllenAtlas
import numpy as np

brain_atlas = AllenAtlas()


# Transform coords from CCF origin (order mlapdv) to Bregma origin (order mlapdv)
# ccf coordinates in um
ccf_mlapdv = np.array([[3000, 4000, 3000], [6000, 6000, 500] ], dtype=np.float32)
# returned bregma coordinates in m
bregma_mlapdv = brain_atlas.ccf2xyz(ccf_mlapdv, ccf_order='mlapdv')


# Transform coords from CCF origin (order apdvml) to Bregma origin (order mlapdv)
# ccf coordinates in um
ccf_apdvml = np.array([[3000, 4000, 3000], [6000, 6000, 500] ], dtype=np.float32)
# returned bregma coordinates in m
bregma_mlapdv = brain_atlas.ccf2xyz(ccf_apdvml, ccf_order='apdvml')

If you want to transform the other way, from bregma coordinates to CCF coordinates, an inverse function is also available

In [2]:
# Transform coords from Bregma origin (order mlapdv) to CCF origin (order mlapdv)
# example coordinates in m with Bregma origin
bregma_mlapdv = np.array([[2000, 4000, 0], [4000, -1000, -4000]]) / 1e6 
ccf_mlapdv = brain_atlas.xyz2ccf(bregma_mlapdv, ccf_order='mlapdv')

# Transform coords from Bregma origin (order mlapdv) to CCF origin (order apdvml)
bregma_mlapdv = np.array([[2000, 4000, 0], [4000, -1000, -4000]]) / 1e6
ccf_apdvml = brain_atlas.xyz2ccf(bregma_mlapdv, ccf_order='apdvml')

## Uploading an example points

Here we show an example of uploading a set of points to both a local and remote bucket. To understand the difference between the two, please refer to the [getting started tutorial](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_getting_started.ipynb#3.-Understand-some-useful-concepts).

In [4]:
import numpy as np
# Define an array of 3D points
# Rows are individual points, coordinates per row are [ml, ap, dv] in um relative to bregma
xyz = np.array([[-2663, -2624, -118],
                [-2338, -2650, -1193],
                [-2289, -2650, -1592],
                [-2214, -2650, -1993],
                [-2263, -2775, -2193],
                [-2163, -2650, -2368],
                [-1988, -2775, -3493],
                [-1963, -2775, -3892],
                [-1863, -2775, -4343],
                [-1788, -2775, -4792]])

# Convert to metres
xyz = xyz / 1e6

# Create a feature with random values between 0 - 100, one value per point
values = np.random.randint(0, 100, xyz.shape[0])

In [5]:
from iblbrainviewer.api import FeatureUploader
from pathlib import Path

# Instantiate local bucket
local_bucket_path = Path.home().joinpath('local_bucket')
up_local = FeatureUploader()

# Instantiate remote bucket
bucket = 'ibl_points_example'
up_remote = FeatureUploader(bucket)



# Upload locally
# Upload points with a size of 3 (default)
fname = 'points'
up_local.local_dots(fname, xyz, values, dot_size=3, output_dir=local_bucket_path)

fname = 'points'
# Upload remotely
if not up_remote.features_exist(fname):
    # For acronyms must provide hemisphere argument
    up_remote.upload_dots(fname, xyz, values, dot_size=3)

url = up_remote.get_buckets_url([bucket])
print(url)

https://atlas.internationalbrainlab.org/?buckets=ibl_points_example&bucket=ibl_points_example
